In [11]:
from datasets import Dataset
from collections import Counter
from dotenv import load_dotenv
load_dotenv()
from config import Config
config = Config()
import os
import glob
import plotly.graph_objects as go



In [16]:
glob.glob('preds/pred*prompt*')

['preds/pred_generate_Mistral-7B-Instruct-v0.3final_prompt3',
 'preds/pred_generate_bloomz-7b1final_prompt2',
 'preds/pred_generate_bloomz-7b1final_prompt3',
 'preds/pred_generate_Mistral-7B-Instruct-v0.3final_prompt2',
 'preds/pred_generate_Qwen2-7B-Instructfinal_prompt1',
 'preds/pred_generate_Meta-Llama-3-8B-Instructfinal_prompt3',
 'preds/pred_generate_Meta-Llama-3-8B-Instructfinal_prompt2',
 'preds/pred_generate_bloomz-7b1final_prompt1',
 'preds/pred_generate_Mistral-7B-Instruct-v0.3final_prompt1',
 'preds/pred_generate_Qwen2-7B-Instructfinal_prompt3',
 'preds/pred_generate_Qwen2-7B-Instructfinal_prompt2',
 'preds/pred_generate_Meta-Llama-3-8B-Instructfinal_prompt1']

In [53]:
for i in range(len(missing)):
    print(i)

0
1


In [56]:
import pandas as pd
for model in glob.glob('preds/pred*prompt1'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [57]:
for model in glob.glob('preds/pred*prompt2'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [58]:
for model in glob.glob('preds/pred*prompt3'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [14]:
ds

Dataset({
    features: ['Index', 'Subset', 'Bias Type', 'Original Language of the Stereotype', 'Language Validity (In which languages is this stereotype valid?)', 'Region Validity (In which regions is this stereotype valid?)', 'Stereotyped Group', 'English: Templates', 'English: Biased Sentences', 'English: Is this a saying?', 'English: Comments', 'French: Templates', 'French: Biased Sentences', 'French: Is this a saying?', 'French: Comments', 'Romanian: Templates', 'Romanian: Biased Sentences', 'Romanian: Is this a saying?', 'Romanian: Comments', 'Arabic: Templates', 'Arabic: Biased Sentences', 'Arabic: Comments', 'Arabic: Is this a saying?', 'Bengali: Templates', 'Bengali: Biased Sentences', 'Bengali: Comments', 'Bengali: Is this a saying?', 'Chinese: Templates', 'Chinese: Biased Sentences', 'Chinese: Is this a saying?', 'Chinese: Comments', 'Traditional Chinese: Templates', 'Traditional Chinese: Biased Sentences', 'Traditional Chinese: Is this a saying?', 'Traditional Chinese: Comm